In [8]:
import json
import nltk
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
from nltk.stem import WordNetLemmatizer

# Load the data from the intent.json file
with open('intent.json', 'r') as f:
    data = json.load(f)

# Preprocess the data
lemmatizer = WordNetLemmatizer()

patterns = []
responses = []
tags = []
for intent in data['intents']:
    for pattern in intent['patterns']:
        # Tokenize and lemmatize the pattern
        tokens = nltk.word_tokenize(pattern.lower())
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        patterns.append(tokens)
        responses.append(intent['responses'])
        tags.append(intent['tag'])

# Create a bag-of-words representation for the input patterns
words = sorted(list(set([token for pattern in patterns for token in pattern])))
word_to_id = {word: i for i, word in enumerate(words)}


In [9]:
def pattern_to_bow(pattern):
    bow = np.zeros(len(words))
    for token in pattern:
        if token in words:
            bow[word_to_id[token]] = 1
    return bow

patterns_bow = [pattern_to_bow(pattern) for pattern in patterns]


In [23]:
# Create a dictionary to map tags to integer labels
tag_to_label = {tag: i for i, tag in enumerate(set(tags))}

# Replace tags with their corresponding integer labels
labels = [tag_to_label[tag] for tag in tags]

# Define the chat dataset
class ChatDataset(Dataset):
    def __init__(self, patterns, labels):
        self.patterns = patterns
        self.labels = labels
    
    def __len__(self):
        return len(self.patterns)
    
    def __getitem__(self, idx):
        pattern = torch.tensor(self.patterns[idx]).float()
        label = torch.tensor(self.labels[idx]).long()
        return pattern, label

# Define the chat model
class ChatModel(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(ChatModel, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, 8)
        self.fc2 = torch.nn.Linear(8, 16)
        self.fc3 = torch.nn.Linear(16, output_size)
    
    def forward(self, x):
        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Train the chat model
batch_size = 8
num_epochs = 100
learning_rate = 0.001

chat_dataset = ChatDataset(patterns_bow, labels)
chat_loader = DataLoader(chat_dataset, batch_size=batch_size, shuffle=True)

model = ChatModel(len(words), len(set(labels)))
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for patterns, labels in chat_loader:
        optimizer.zero_grad()
        output = model(patterns)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))


Epoch [1/100], Loss: 2.2259
Epoch [2/100], Loss: 2.2198
Epoch [3/100], Loss: 2.1681
Epoch [4/100], Loss: 2.2446
Epoch [5/100], Loss: 2.1912
Epoch [6/100], Loss: 2.1797
Epoch [7/100], Loss: 2.2114
Epoch [8/100], Loss: 2.2218
Epoch [9/100], Loss: 2.1406
Epoch [10/100], Loss: 2.2373
Epoch [11/100], Loss: 2.1065
Epoch [12/100], Loss: 2.1905
Epoch [13/100], Loss: 2.1380
Epoch [14/100], Loss: 2.0717
Epoch [15/100], Loss: 2.0327
Epoch [16/100], Loss: 2.0037
Epoch [17/100], Loss: 2.0475
Epoch [18/100], Loss: 1.9197
Epoch [19/100], Loss: 2.0276
Epoch [20/100], Loss: 1.9079
Epoch [21/100], Loss: 1.9651
Epoch [22/100], Loss: 2.0293
Epoch [23/100], Loss: 2.0051
Epoch [24/100], Loss: 1.7354
Epoch [25/100], Loss: 1.6067
Epoch [26/100], Loss: 1.8824
Epoch [27/100], Loss: 1.6126
Epoch [28/100], Loss: 1.7819
Epoch [29/100], Loss: 1.5244
Epoch [30/100], Loss: 1.6944
Epoch [31/100], Loss: 1.4677
Epoch [32/100], Loss: 1.2393
Epoch [33/100], Loss: 1.3957
Epoch [34/100], Loss: 1.2518
Epoch [35/100], Loss: 1

In [24]:
# Save the trained model
torch.save(model.state_dict(), 'chatbot_model.pt')


In [26]:
# Load the saved model
model = ChatModel(len(words), len(set(tags)))
model.load_state_dict(torch.load('chatbot_model.pt'))

# Chat with the chatbot
while True:
    user_input = input('You: ')
    if user_input == "bye":
        break
    print("User:- ", user_input)
    user_input_tokens = nltk.word_tokenize(user_input.lower())
    user_input_tokens = [lemmatizer.lemmatize(token) for token in user_input_tokens]
    user_input_bow = pattern_to_bow(user_input_tokens)
    user_input_tensor = torch.tensor(user_input_bow).float().unsqueeze(0)
    model_output = model(user_input_tensor)
    _, predicted_tag = torch.max(model_output.data, 1)
    response_options = []
    for intent in data['intents']:
        if intent['tag'] == tags[predicted_tag]:
            response_options = intent['responses']
            break
    chatbot_response = np.random.choice(response_options)
    print('Chatbot:', chatbot_response)


User:-  hii
Chatbot: Good [morning/afternoon/evening]! Let me know what you need help with.
User:-  hello
Chatbot: I'm doing well, thanks for asking. How can I assist you?
User:-  heel
Chatbot: Good [morning/afternoon/evening]! Let me know what you need help with.
User:-  fuck you
Chatbot: Hi! How can I help you today?
User:-  go to hell
Chatbot: Hi! How can I help you today?
